In [ ]:
import math, re, os
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from kaggle_datasets import KaggleDatasets
from tensorflow import keras
from functools import partial
from sklearn.model_selection import train_test_split
from kaggle_datasets import KaggleDatasets
print("Tensorflow version " + tf.__version__)
# Keras Core
from keras.layers.convolutional import MaxPooling2D, Convolution2D, AveragePooling2D
from keras.layers import Input, Dropout, Dense, Flatten, Activation
from keras.layers.normalization import BatchNormalization
from keras.layers.merge import concatenate
from keras import regularizers
from keras import initializers
from keras.models import Model
# Backend
from keras import backend as K
from keras.applications import imagenet_utils
from keras import layers
import tensorflow_addons as tfa
from tqdm import tqdm

In [ ]:
IMAGE_SIZE = 600


In [ ]:
def auto_select_accelerator():
    """
    Reference: 
        * https://www.kaggle.com/mgornergoogle/getting-started-with-100-flowers-on-tpu
        * https://www.kaggle.com/xhlulu/ranzcr-efficientnet-tpu-training
    """
    try:
        tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
        tf.config.experimental_connect_to_cluster(tpu)
        tf.tpu.experimental.initialize_tpu_system(tpu)
        strategy = tf.distribute.experimental.TPUStrategy(tpu)
        print("Running on TPU:", tpu.master())
    except ValueError:
        strategy = tf.distribute.get_strategy()
    print(f"Running on {strategy.num_replicas_in_sync} replicas")
    return strategy



def build_decoder(with_labels=True, target_size=(IMAGE_SIZE, IMAGE_SIZE), ext='jpg'):
    def decode(path):
        file_bytes = tf.io.read_file(path)
        if ext == 'png':
            img = tf.image.decode_png(file_bytes, channels=3)
        elif ext in ['jpg', 'jpeg']:
            img = tf.image.decode_jpeg(file_bytes, channels=3)
        else:
            raise ValueError("Image extension not supported")

        img = tf.cast(img, tf.float32) / 255.0
        img = tf.image.resize(img, target_size)

        return img
    
    def decode_with_labels(path, label):
        return decode(path), label
    
    return decode_with_labels if with_labels else decode


def build_augmenter(img, label):
    #seed = RNG.make_seeds(2)[0]

    def augment(img):
        img = tf.image.resize_with_crop_or_pad(img, IMAGE_SIZE + 6, IMAGE_SIZE + 6)
        #tf.print("seed is:", seed)
        # Random crop back to the original size
        img = tf.image.random_crop(img, size=[IMAGE_SIZE, IMAGE_SIZE, 3])
        img = tf.image.random_flip_left_right(img)
        img = tf.image.random_flip_up_down(img)
        img = tf.image.random_contrast(img, lower=0.95, upper=1.1)
        img = tf.image.random_brightness(img, max_delta=0.05)
        img = tf.image.random_saturation(img, 0.95, 1.05)
        img = tf.image.random_jpeg_quality(img, 90, 100)
        img = tf.clip_by_value(img, 0, 1)
        img = tf.image.resize(img, (IMAGE_SIZE, IMAGE_SIZE))
        return img
                
    return augment(img), label


def build_dataset(paths, labels=None, bsize=32, cache=True,
                  decode_fn=None, augment_fn=build_augmenter,
                  augment=True, repeat=True, shuffle=1024, 
                  cache_dir=""):
    if cache_dir != "" and cache is True:
        os.makedirs(cache_dir, exist_ok=True)

    
    AUTO = tf.data.experimental.AUTOTUNE
    slices = paths if labels is None else (paths, labels)
    
    dset = tf.data.Dataset.from_tensor_slices(slices)
    dset = dset.map(decode_fn, num_parallel_calls=AUTO)
    dset = dset.cache(cache_dir) if cache else dset
    if augment:
        dset = dset.map(augment_fn, num_parallel_calls=AUTO) 
    dset = dset.repeat() if repeat else dset
    dset = dset.shuffle(shuffle) if shuffle else dset
    dset = dset.batch(bsize).prefetch(AUTO)
    
    return dset

In [ ]:

COMPETITION_NAME = "ranzcr-clip-catheter-line-classification"
strategy = auto_select_accelerator()
PER_REPLICA_BATCH_SIZE = 16
GLOBAL_BATCH_SIZE = strategy.num_replicas_in_sync * PER_REPLICA_BATCH_SIZE
GCS_DS_PATH = KaggleDatasets().get_gcs_path(COMPETITION_NAME)


In [ ]:
def get_train_datasets(curr_fold, batch_size):
    load_dir = f"/kaggle/input/{COMPETITION_NAME}/"
    df = pd.read_csv("../input/ranzcr-clip-catheter-line-classification/train.csv") 

    paths = GCS_DS_PATH + "/train/" + df['StudyInstanceUID'] + '.jpg'

    sub_df = pd.read_csv(load_dir + 'sample_submission.csv')

    test_paths = GCS_DS_PATH + "/test/" + sub_df['StudyInstanceUID'] + '.jpg'

    # Get the multi-labels
    label_cols = sub_df.columns[1:]
    labels = df[label_cols]


    df = pd.read_csv("../input/how-to-properly-split-folds/train_folds.csv") 
    valid_idx = df["fold"] == curr_fold 

    train_paths = paths[~valid_idx].values.tolist()
    train_labels = labels[~valid_idx].values.tolist()

    valid_paths = paths[valid_idx].values.tolist()
    valid_labels = labels[valid_idx].values.tolist()
    

    decoder = build_decoder(with_labels=True, target_size=(IMAGE_SIZE, IMAGE_SIZE))
    test_decoder = build_decoder(with_labels=False, target_size=(IMAGE_SIZE, IMAGE_SIZE))

    train_dataset = build_dataset(
        train_paths, train_labels, bsize=batch_size, decode_fn=decoder, cache=True
    )

    valid_dataset = build_dataset(
        valid_paths, valid_labels, bsize=batch_size, decode_fn=decoder, cache=True,
            repeat=False, shuffle=False, augment=False
    )
    
    size_train_dataset = len(train_paths)
    return train_dataset, valid_dataset, size_train_dataset


# MODEL

Model from paper "Inception-v4, Inception-ResNet and the Impact of Residual Connections on Learning" https://arxiv.org/pdf/1602.07261.pdf


In [ ]:
BATCH_NORM_EPSILON = 0.001
BATCH_NORM_DECAY = 0.99
WEIGHT_DECAY = 1e-6

In [ ]:

#function from https://github.com/keras-team/keras/blob/master/keras/applications/inception_v3.py
"""Utility function to apply conv + BN. 
  Args:
    x: input tensor.
    filters: filters in `Conv2D`.
    num_row: height of the convolution kernel.
    num_col: width of the convolution kernel.
    padding: padding mode in `Conv2D`.
    strides: strides in `Conv2D`.
    name: name of the ops; will become `name + '_conv'`
      for the convolution and `name + '_bn'` for the
      batch norm layer.
  Returns:
    Output tensor after applying `Conv2D` and `BatchNormalization`.
  """
def conv2d_bn(x, filters, num_row, num_col, padding='same', strides=(1, 1), name=None):
    if name is not None:
        bn_name = name + '_bn'
        conv_name = name + '_conv'
    else:
        bn_name = None
        conv_name = None
    if K.image_data_format() == 'channels_first':
        bn_axis = 1
    else:
        bn_axis = 3
    x = layers.Conv2D(
        filters, (num_row, num_col),
        strides=strides,
        padding=padding,
        use_bias=False,
        kernel_regularizer = tf.keras.regularizers.L2(l2 = WEIGHT_DECAY),
        name=conv_name)(
          x)
    x = tf.keras.layers.experimental.SyncBatchNormalization(
                                  axis=bn_axis,
                                  momentum = BATCH_NORM_DECAY,
                                  epsilon = BATCH_NORM_EPSILON,
                                  scale=False,
                                  name=bn_name)(x)
    
    x = layers.Activation('relu', name=name)(x)
    return x



In [ ]:
def stem(x):
    # shape of x is 229 x 229 x 3 (channel first)
    # shape of x is 3 x 229 x 229 (channel last)
    if K.image_data_format() == 'channels_first':
        channel_axis = 1
    else:
        channel_axis = -1
        
    x = conv2d_bn(x, 32, 3, 3, padding="valid", strides=(2,2))
    x = conv2d_bn(x, 32, 3, 3, padding="valid")
    x = conv2d_bn(x, 64, 3, 3)
    
    branch_0 = MaxPooling2D((3,3),strides=(2,2),padding="valid")(x)
    branch_1 = conv2d_bn(x, 96, 3, 3, strides=(2,2),padding="valid")
    
    x = concatenate([branch_0, branch_1], axis=channel_axis)
    
    branch_0 = conv2d_bn(x, 64, 1, 1)
    branch_0 = conv2d_bn(branch_0, 96, 3, 3, padding="valid")
    
    branch_1 = conv2d_bn(x, 64, 1, 1)
    branch_1 = conv2d_bn(branch_1, 64, 7, 1)
    branch_1 = conv2d_bn(branch_1, 64, 1, 7)
    branch_1 = conv2d_bn(branch_1, 96, 3, 3, padding="valid")
    
    x = concatenate([branch_0, branch_1], axis=channel_axis)
    
    branch_0 = conv2d_bn(x, 192, 3, 3, strides=(2,2), padding="valid")
    branch_1 = MaxPooling2D((3,3),strides=(2,2),padding="valid")(x)
    
    x = concatenate([branch_0, branch_1], axis=channel_axis)
    return x



In [ ]:
def inception_a(x):
    if K.image_data_format() == 'channels_first':
        channel_axis = 1
    else:
        channel_axis = -1
        
    branch_0 = AveragePooling2D((3,3), strides=(1,1), padding="same")(x)
    branch_0 = conv2d_bn(branch_0, 96, 1, 1)
    
    branch_1 = conv2d_bn(x, 96, 1, 1)
    
    branch_2 = conv2d_bn(x, 64, 1, 1)
    branch_2 = conv2d_bn(branch_2, 96, 3, 3)

    branch_3 = conv2d_bn(x, 64, 1, 1)
    branch_3 = conv2d_bn(branch_3, 96, 3, 3)
    branch_3 = conv2d_bn(branch_3, 96, 3, 3)
    
    x = concatenate([branch_0, branch_1, branch_2, branch_3], axis=channel_axis)
    return x


In [ ]:
def reduction_a(x):
    if K.image_data_format() == 'channels_first':
        channel_axis = 1
    else:
        channel_axis = -1
    
    branch_0 = MaxPooling2D((3,3), strides=(2,2), padding="valid")(x)
    
    branch_1 = conv2d_bn(x, 384, 3, 3, strides=(2,2), padding="valid")
    
    branch_2 = conv2d_bn(x, 192, 1, 1)
    branch_2 = conv2d_bn(branch_2, 224, 3, 3)
    branch_2 = conv2d_bn(branch_2, 256, 3, 3, strides=(2,2), padding="valid")
    
    x = concatenate([branch_0, branch_1, branch_2], axis=channel_axis)
    return x

    

In [ ]:
def inception_b(x):
    if K.image_data_format() == 'channels_first':
        channel_axis = 1
    else:
        channel_axis = -1
    branch_0 = AveragePooling2D((3,3), strides=(1,1), padding="same")(x)
    branch_0 = conv2d_bn(branch_0, 128, 1, 1)
    
    branch_1 = conv2d_bn(x, 384, 1, 1)
    
    branch_2 = conv2d_bn(x, 192, 1, 1)
    branch_2 = conv2d_bn(branch_2, 224, 1, 7)
    branch_2 = conv2d_bn(branch_2, 256, 7, 1)
    
    branch_3 = conv2d_bn(x, 192, 1, 1)
    branch_3 = conv2d_bn(branch_3, 192, 1, 7)
    branch_3 = conv2d_bn(branch_3, 224, 7, 1)
    branch_3 = conv2d_bn(branch_3, 224, 1, 7)
    branch_3 = conv2d_bn(branch_3, 256, 7, 1)
    
    x = concatenate([branch_0, branch_1, branch_2, branch_3], axis=channel_axis)
    return x

In [ ]:
def reduction_b(x):
    if K.image_data_format() == 'channels_first':
        channel_axis = 1
    else:
        channel_axis = -1
    branch_0 = MaxPooling2D((3,3), strides=(2,2), padding="valid")(x)
    
    branch_1 = conv2d_bn(x, 192, 1, 1)
    branch_1 = conv2d_bn(branch_1, 192, 3, 3, strides=(2,2), padding="valid")
    
    branch_2 = conv2d_bn(x, 256, 1, 1)
    branch_2 = conv2d_bn(branch_2, 256, 1, 7)
    branch_2 = conv2d_bn(branch_2, 320, 7, 1)
    branch_2 = conv2d_bn(branch_2, 320, 3, 3, strides=(2,2), padding="valid")
    
    x = concatenate([branch_0, branch_1, branch_2], channel_axis)
    return x


In [ ]:
def inception_c(x):
    if K.image_data_format() == 'channels_first':
        channel_axis = 1
    else:
        channel_axis = -1
        
    branch_0 = AveragePooling2D((3,3), strides=(1,1), padding="same")(x)
    branch_0 = conv2d_bn(branch_0, 256, 1, 1)
    
    branch_1 = conv2d_bn(x, 256, 1, 1)
    
    branch_2 = conv2d_bn(x, 384, 1, 1)
    branch_2a = conv2d_bn(branch_2, 256, 1, 3)
    branch_2b = conv2d_bn(branch_2, 256, 3, 1)
    
    branch_3 = conv2d_bn(x, 384, 1, 1)
    branch_3 = conv2d_bn(branch_3, 448, 1, 3)
    branch_3 = conv2d_bn(branch_3, 512, 3, 1)
    branch_3a = conv2d_bn(branch_3, 256, 1, 3)
    branch_3b = conv2d_bn(branch_3, 256, 3, 1)
    
    x = concatenate([branch_0, branch_1, branch_2a,
                     branch_2b, branch_3a, branch_3b], channel_axis)
    return x

In [ ]:
def InceptionV4(input_tensor=None, input_shape=None, classes=1000):
      #input_tensor: Optional Keras tensor (i.e. output of `layers.Input()`)
      #to use as image input for the model. `input_tensor` is useful for sharing
      #inputs between multiple different networks. Default to None.
    
    
    if input_tensor is None:
        img_input = layers.Input(shape=input_shape)
    
    else:
        if not K.is_keras_tensor(input_tensor):
            img_input = layers.Input(tensor=input_tensor, shape=input_shape)
        else:
            img_input = input_tensor
            
    if K.image_data_format() == 'channels_first':
        channel_axis = 1
    else:
        channel_axis = 3
        
    
    x = stem(img_input)
    for i in range(4):
        x = inception_a(x)
        
    x = reduction_a(x)
    for i in range(7):
        x = inception_b(x)
        
    x = reduction_b(x)
    for i in range(3):
        x = inception_c(x)
    
    x = layers.GlobalAveragePooling2D()(x)
    x = Dropout(rate=0.2)(x)
    
    x = Dense(classes,
              kernel_regularizer = tf.keras.regularizers.L2(l2 = WEIGHT_DECAY),
              activation="sigmoid")(x)
    
    # Ensure that the model takes into account
    # any potential predecessors of `input_tensor`.
    if input_tensor is not None:
        inputs = layer_utils.get_source_inputs(input_tensor)
    else:
        inputs = img_input
    # Create model.
    model = Model(inputs, x, name='inception_v4')
    return model
    

# TRAINING


In [ ]:
# reduces the learning rate of the optimizer by factor after "patience" epochs
# in which the validation AUC has not improved
class ReduceLRScheduler():
    def __init__(self, optimizer, patience, factor, min_lr):
        self.optimizer = optimizer
        self.patience = patience
        self.factor = factor
        self.min_lr = min_lr
        self.best_auc = None
        self.epochs_no_improvement = 0
    
    def on_epoch_end(self, val_auc):
        # called first epoch
        if self.best_auc is None:
            self.best_auc = val_auc
        # improvement
        elif val_auc > self.best_auc:
            self.epochs_no_improvement = 0
        # no improvement
        else:
            self.epochs_no_improvement += 1
            if self.epochs_no_improvement >= self.patience:
                self.optimizer.learning_rate = max(self.min_lr, self.optimizer.learning_rate * self.factor)
                print("reducing learning rate to:", self.optimizer.learning_rate)
        

class EarlyStopping():
    def __init__(self, patience):
        self.epochs_no_improvement = 0
        self.best_auc = None
        self.patience = patience
        
    def hasToStop(self, val_auc):
        if self.best_auc is None:
            self.best_auc = val_auc
            return False
        
        elif val_auc > self.best_auc:
            self.best_auc = val_auc
            self.epochs_no_improvement = 0
            return False
        
        else:
            self.epochs_no_improvement += 1
            return self.epochs_no_improvement >= self.patience
        
    

In [ ]:
##### TODO: Early Stopping Epochs no improv = 15
#####       change number of epochs to 200
#####   try removing weight decay..?
#####   put a safety condition: if val_auc < 1e-3: interrupt training
#####   to not waste TPU

In [ ]:
def train_fold(n_fold):
    n_labels = 11
    tf.keras.backend.clear_session() #clear memory
    
    train_dataset, valid_dataset, size_train_dataset = get_train_datasets(curr_fold=n_fold, batch_size=GLOBAL_BATCH_SIZE)
    
    train_dist_dataset = strategy.experimental_distribute_dataset(train_dataset)
    valid_dist_dataset = strategy.experimental_distribute_dataset(valid_dataset)
    
    steps_per_epoch = size_train_dataset // GLOBAL_BATCH_SIZE


    EPOCHS = 200

    
    with strategy.scope():
        ############ DEFINE LOSS ####################################
        # Set reduction to `none` so we can do the reduction afterwards and divide by
        # global batch size.
        loss_object = tf.keras.losses.BinaryCrossentropy(reduction=tf.keras.losses.Reduction.NONE)
        def compute_loss(model, labels, predictions):
            per_example_loss = loss_object(labels, predictions)

            # Compute loss that is scaled by sample_weight and by global batch size.
            loss = tf.nn.compute_average_loss(
                                per_example_loss,
                                global_batch_size=GLOBAL_BATCH_SIZE)

            # Add scaled regularization losses.
            loss += tf.nn.scale_regularization_loss(model.losses)
            return loss
        
        ############ METRICS #########################
        train_auc = tf.keras.metrics.AUC(name="train_auc", multi_label=True)
        val_auc = tf.keras.metrics.AUC(name="val_auc", multi_label=True)
        
        ############ MODEL #######################
        
        model = InceptionV4(input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3), classes = n_labels)

        optimizer = tf.keras.optimizers.RMSprop(
                    learning_rate=0.001, rho=0.9, momentum=0.9, epsilon=1e-7, centered=False)
    
        optimizer  = tfa.optimizers.MovingAverage(optimizer)
        
        lr_scheduler = ReduceLRScheduler(optimizer, patience=3, factor=5, min_lr=1e-6)
        early_stopping = EarlyStopping(patience = 15)
        
    ############ TRAINING LOOP #######################
        
    def train_step(inputs):
        images, labels = inputs
        with tf.GradientTape() as tape:
            predictions = model(images, training=True)
            loss = compute_loss(model, labels, predictions)
        
        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))
        
        train_auc.update_state(labels, predictions)
        return loss
    
    def test_step(inputs):
        images, labels = inputs
        predictions = model(images, training=False)
        loss = compute_loss(model, labels, predictions)
        val_auc.update_state(labels, predictions)
        return loss
        
    # `run` replicates the provided computation and runs it
    # with the distributed input.
    @tf.function
    def distributed_train_step(dataset_inputs):
        per_replica_losses = strategy.run(train_step, args=(dataset_inputs,))
        return strategy.reduce(tf.distribute.ReduceOp.SUM, per_replica_losses,
                         axis=None)

    @tf.function
    def distributed_test_step(dataset_inputs):
        per_replica_losses = strategy.run(test_step, args=(dataset_inputs,))
        return strategy.reduce(tf.distribute.ReduceOp.SUM, per_replica_losses,
                         axis=None)
    
    val_aucs = []
    train_aucs = []

    for epoch in range(EPOCHS):
        print("Fold:", n_fold, "Epoch:", epoch+1, "Training")
        
        ######## TRAIN LOOP ###############
        total_loss = 0.0
        num_batches = 0
        train_iter = iter(train_dist_dataset)
        for i in tqdm(range(steps_per_epoch)):    
            total_loss += distributed_train_step(next(train_iter))
            num_batches += 1

        train_loss = total_loss / num_batches
        
        ######## TEST LOOP ###############
        # loading averaged weights
        with strategy.scope():
            optimizer.swap_weights()
            

        total_loss = 0.0
        num_batches = 0
        for x in valid_dist_dataset:
            total_loss += distributed_test_step(x)
            num_batches += 1
            
        val_loss = total_loss / num_batches
            
        # loading back original weights
        with strategy.scope():
            optimizer.swap_weights()
        
        template = ("Epoch: {}, Loss: {:.4f}, train_auc: {:.4f}, val_loss: {:.4f}, val_auc: {:.4f}")
        print(template.format(epoch+1, train_loss, 
                              train_auc.result(), val_loss, val_auc.result()))
        
        val_aucs.append(tf.cast(val_auc.result(), tf.float32))
        train_aucs.append(tf.cast(train_auc.result(), tf.float32))
        
        ############ SAVE BEST MODEL ##############
        if val_auc.result() >= max(val_aucs):
            print("Saving best model with validation AUC:", str(tf.cast(val_auc.result(), tf.float32)))
            with strategy.scope():
                optimizer.swap_weights()
            model.save("model_fold_" + str(n_fold) + ".h5")
            with strategy.scope():
                optimizer.swap_weights()
        
        # Sanity Check, problems due to weight regularization
        if val_auc.result() < 1e-3:
            print("Numerical problem... Validation AUC is too low. Interrupting training")
            break
            
        # CallBacks 
        if early_stopping.hasToStop(val_auc.result()):
            print("No improvement after", early_stopping.patience, "epochs. Interrupting training")
            break
            
        lr_scheduler.on_epoch_end(val_auc.result())
        
        train_auc.reset_states()
        val_auc.reset_states()
        
    data = {"val_auc": val_aucs, "train_auc" : train_aucs}
    history = pd.DataFrame(data=data)
    history.to_csv("history_" + str(n_fold) + ".csv")

In [ ]:
def train_folds():
    for i in range(5):
        train_fold(n_fold=i)


In [ ]:
train_fold(0)